In [2]:
import json

from collections import defaultdict

data_path = './data/'

In [4]:
# 词表大小与未登录词，先看标题 & 摘要

In [8]:
with open('{}/train_large/news.tsv'.format(data_path)) as f:
    for line in f:
        break

In [10]:
news_id, category, subcategory, title, abstract, url, title_entities, abstract_entities = line.replace('\n', '').split('\t')

In [25]:
print(news_id)
print(category)
print(subcategory)
print(title)
print(abstract)
print(url)
print(json.loads(title_entities))
print(json.loads(abstract_entities))

N88753
lifestyle
lifestyleroyals
The Brands Queen Elizabeth, Prince Charles, and Prince Philip Swear By
Shop the notebooks, jackets, and more that the royals can't live without.
https://assets.msn.com/labs/mind/AAGH0ET.html
[{'Label': 'Prince Philip, Duke of Edinburgh', 'Type': 'P', 'WikidataId': 'Q80976', 'Confidence': 1.0, 'OccurrenceOffsets': [48], 'SurfaceForms': ['Prince Philip']}, {'Label': 'Charles, Prince of Wales', 'Type': 'P', 'WikidataId': 'Q43274', 'Confidence': 1.0, 'OccurrenceOffsets': [28], 'SurfaceForms': ['Prince Charles']}, {'Label': 'Elizabeth II', 'Type': 'P', 'WikidataId': 'Q9682', 'Confidence': 0.97, 'OccurrenceOffsets': [11], 'SurfaceForms': ['Queen Elizabeth']}]
[]


In [37]:
train_vocab = defaultdict(int)
test_vocab = defaultdict(int)

In [38]:
with open('{}/data_large/train/news.tsv'.format(data_path), encoding='utf-8') as f:
    for line in f:
        _, _, _, title, abstract, _, _, _ = line.replace('\n', '').split('\t')
        for word in title.split():
            train_vocab[word] += 1
        for word in abstract.split():
            train_vocab[word] += 1

In [39]:
with open('{}/test/news.tsv'.format(data_path), encoding='utf-8') as f:
    for line in f:
        _, _, _, title, abstract, _, _, _ = line.replace('\n', '').split('\t')
        for word in title.split():
            test_vocab[word] += 1
        for word in abstract.split():
            test_vocab[word] += 1

In [40]:
len(train_vocab)

218873

In [41]:
len(test_vocab)

242343

In [45]:
len(set(train_vocab).union(set(test_vocab))) - len(set(train_vocab))   # 3w+ 的未登录词

33951

In [51]:
n = 0
for _, freq in train_vocab.items():
    if freq > 10:
        n += 1

In [52]:
n     # 去掉低频词后词典的大小还可以接受，但是如果再加上正文，可能词典大小还会增大

28820

In [53]:
train_save_words = set([word for word in train_vocab if train_vocab[word] > 10])
test_save_words = set([word for word in test_vocab if test_vocab[word] > 10])

In [55]:
len(train_save_words.union(test_save_words)) - len(test_save_words)    # 去掉低频词后未登录词还是有点多

479

In [56]:
# 尝试 bpe subword，看词典大小和未登录词的变化
# 开源工具 subword-nmt
# 先使用训练集生成词典
# 然后将词典用于验证集和测试集
# 在训练模型前，将训练集、验证集、测试集根据空格 split 后转化为 index 即可

In [58]:
# 前面就把词典和截断的长度定下来，或者说截断长度相同 & 合并次数 & 使用 title/abstract/content 这三个条件都相同的情况下，词典就存储下来
# 每次实验都读取之前的词典，如果出现新的组合，再重新生成词典
# 词典按照最大的训练集生成，在使用demo 和 small测试时也使用最大的词典

In [61]:
# 首先获取新闻的正文
nid2url = {}
with open('{}/data_large/train/news.tsv'.format(data_path), encoding='utf-8') as f:
    for line in f:
        news_id, _, _, _, _, url, _, _ = line.replace('\n', '').split('\t')
        nid2url[news_id] = url
with open('{}/test/news.tsv'.format(data_path), encoding='utf-8') as f:
    for line in f:
        news_id, _, _, _, _, url, _, _ = line.replace('\n', '').split('\t')
        nid2url[news_id] = url
with open('{}/data_large/val/news.tsv'.format(data_path), encoding='utf-8') as f:
    for line in f:
        news_id, _, _, _, _, url, _, _ = line.replace('\n', '').split('\t')
        nid2url[news_id] = url

In [62]:
len(nid2url)  # 13w新闻

130379

In [64]:
with open('{}/news_url'.format(data_path), 'w') as f:
    for news_id, url in nid2url.items():
        f.write('{}\t{}\n'.format(news_id, url))

In [65]:
# 解析方法 https://github.com/msnews/MIND/tree/master/crawler
url

'https://assets.msn.com/labs/mind/BBWyOPQ.html'

In [1]:
import torch

In [2]:
a = torch.tensor(list(range(6))).view(2, 3)

In [6]:
b = torch.tensor([0, 1]).unsqueeze(1)

In [9]:
a

tensor([[0, 1, 2],
        [3, 4, 5]])

In [8]:
a * b

tensor([[0, 0, 0],
        [3, 4, 5]])

In [12]:
torch.linspace(3, 10, 1)

tensor([3.])

In [13]:
import numpy as np

In [15]:
help(np.random.randint)

Help on built-in function randint:

randint(...) method of numpy.random.mtrand.RandomState instance
    randint(low, high=None, size=None, dtype=int)
    
    Return random integers from `low` (inclusive) to `high` (exclusive).
    
    Return random integers from the "discrete uniform" distribution of
    the specified dtype in the "half-open" interval [`low`, `high`). If
    `high` is None (the default), then results are from [0, `low`).
    
    .. note::
        New code should use the ``integers`` method of a ``default_rng()``
        instance instead; please see the :ref:`random-quick-start`.
    
    Parameters
    ----------
    low : int or array-like of ints
        Lowest (signed) integers to be drawn from the distribution (unless
        ``high=None``, in which case this parameter is one above the
        *highest* such integer).
    high : int or array-like of ints, optional
        If provided, one above the largest (signed) integer to be drawn
        from the distributi

In [23]:
np.random.randint(low=0, high=2, size=(2, 3))

array([[0, 1, 1],
       [0, 0, 0]])

In [1]:
# 将新闻正文拼接到 news.tsv 中

In [34]:
def merge_body(path, nid2body):
    with open('{}/news.tsv'.format(path), encoding='utf-8') as f,\
        open('{}/news_with_body.tsv'.format(path), mode='w', encoding='utf-8') as g:
        news_num = 0
        na_num = 0
        for line in f:
            line = line.replace('\n', '')
            _, _, _, _, _, url, _, _ = line.split('\t')
            try:
                uid = url.split('/')[-1].split('.')[-2]
            except Exception as e:
                uid = 'none'
                na_num += 1
            body = nid2body.get(uid, '')
            g.write('{}\t{}\n'.format(line, body))
            news_num += 1
    print(path, news_num, na_num)

In [46]:
paths = [
    './data/data_large/train/',
    './data/data_large/val/',
    './data/test/',
    './data/data_small/train/',
    './data/data_small/val/',
    './data/data_demo/train/',
    './data/data_demo/val/',
]

with open('./data/msn.json') as f:
    nid2body = json.load(f)
nid2body = {i['nid']:' '.join(''.join(i['body']).replace('\xa0', '').split()) for i in nid2body}
# nid2body = {i['nid']:i['body'] for i in nid2body}

In [35]:
for path in paths:
    merge_body(path, nid2body)

./data/data_large/train/ 101527 0
./data/data_large/val/ 72023 0
./data/test/ 120961 0
./data/data_small/train/ 51282 0
./data/data_small/val/ 42416 0
./data/data_demo/train/ 26740 26740
./data/data_demo/val/ 18723 18723


In [ ]:
# .... 可见demo数据集跟large/small不一样，small是从large中抽出来的，demo不是

In [41]:
with open('{}/data_large/train/news_with_body.tsv'.format(data_path), encoding='utf-8') as f:
    for line in f:
        _, _, _, _, _, _, _, _, body = line.replace('\n', '').split('\t')
        break

In [50]:
print(body)

The royals are free to shop wherever they choose, but they tend to keep it in the family—the royal warrant family, that is.The precious few brands with royal warrants are permitted display the Royal Arms in their marketing materials. The royals usually refuse to promote commercial goods, but the warrants are a rare and prominent exception. They're an obvious boon to any branding efforts, but getting your hands on one is no easy task. First, a company must provide goods or services to the royal family (or more specifically, to Queen Elizabeth, Prince Philip, and Prince Charles, the three royals able to award warrants) for five years. Then, as Nigel Fulton, owner of royal warrantee Fulton Umbrellas, told Town & Country earlier this year, "you have to wait until you’re invited to apply, so it’s not automatic at all." To apply, companies need to submit a laundry list of disclosures, which ensure that the business meets various standards. "The finances, your supply base, where all of the co

In [51]:
# 统计加入正文后的词典大小 & 文本长度


In [52]:
train_vocab = defaultdict(int)
test_vocab = defaultdict(int)

with open('./data/data_large/train/news_with_body.tsv', encoding='utf-8') as f:
    for line in f:
        _, _, _, title, abstract, _, _, _, body = line.replace('\n', '').split('\t')
        for word in title.split():
            train_vocab[word] += 1
        for word in abstract.split():
            train_vocab[word] += 1
        for word in body.split():
            train_vocab[word] += 1

with open('./data/test/news_with_body.tsv', encoding='utf-8') as f:
    for line in f:
        _, _, _, title, abstract, _, _, _, body = line.replace('\n', '').split('\t')
        for word in title.split():
            test_vocab[word] += 1
        for word in abstract.split():
            test_vocab[word] += 1
        for word in body.split():
            test_vocab[word] += 1

In [53]:
len(train_vocab)

1763223

In [55]:
len(test_vocab)

1996238

In [64]:
n = 0
for _, freq in train_vocab.items():
    if freq > 100:
        n += 1

In [66]:
n   # 词频大于100的在3w左右

28701

In [67]:
train_vocab = {k:v for k, v in train_vocab.items() if v >= 100}

In [68]:
test_vocab = {k:v for k, v in test_vocab.items() if v >= 100}

In [69]:
len(test_vocab)   # 测试集中依然有很多未登录词

32607

In [70]:
# 尝试 bpe subword，看看未登录词的现象能不能缓解

In [3]:
with open('./data/data_large/train/news_with_body.tsv', encoding='utf-8') as f,\
    open('./data/bpe_corpus', mode='w', encoding='utf-8') as g:
    for line in f:
        _, _, _, title, abstract, _, _, _, body = line.replace('\n', '').split('\t')
        g.write(title + ' ' + abstract + ' ' + body + '\n')